In [36]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from scipy.sparse import save_npz, csc_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import scale

### Смотрим на данные

In [2]:
data1 = pd.read_csv('lst_announcement_data_1.gzip', compression='gzip')
#data2 = pd.read_csv('lst_announcement_data_2.gzip', compression='gzip')

In [3]:
print(data1.shape)
data1.head()

(1000000, 35)


,id,floornumber,category,bargainterms,description,totalarea,geo,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasgarage,hasshower,bedroomscount,hasdishwasher,repairtype,petsallowed,hasbathhouse,hasfridge,wclocationtype,kitchenarea,roomarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,ptn_dadd
0,243455495,NaN,garageSale,"{""price"":3700000.0,""currency"":""rur"",""priceType...",В районе улицы Старый Арбат в клубном доме пре...,12.5,"{""countryId"":138,""undergrounds"":[{""transportTy...","{""parking"":{""type"":""underground""},""cranageType...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-26
1,270527203,2.0,dailyRoomRent,"{""price"":500.0,""currency"":""rur"",""deposit"":0,""b...",Сдается посуточно комфортабельный номер (18м2)...,NaN,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""monolithBrick"",""parking"":{""ty...",{},True,6.0,True,NaN,True,NaN,NaN,NaN,True,NaN,NaN,design,NaN,NaN,True,NaN,NaN,18.0,NaN,NaN,1.0,NaN,NaN,rooms,1.0,2022-06-26
2,270569384,7.0,flatRent,"{""clientFee"":35,""leaseTermType"":""longTerm"",""pr...",Сдам квартиру в хорошем состоянии,38.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""cargoLiftsCount"":0,""materialType"":""panel"",""h...",{},True,1.0,NaN,True,NaN,True,NaN,NaN,NaN,NaN,NaN,cosmetic,False,NaN,True,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,2022-06-26
3,271459928,4.0,flatRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...",Сдам новую 1-комнатную квартиру в Болховском м...,42.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""name"":""ЖК «Мкр. Болховский»"",""parking"":{},""f...",{},NaN,1.0,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,cosmetic,NaN,NaN,True,NaN,NaN,NaN,False,NaN,1.0,NaN,NaN,rooms,NaN,2022-06-26
4,272357233,NaN,houseSale,"{""price"":3500000.0,""currency"":""rur"",""priceType...","Продам дачу в снт Тепличный-1, участок площадь...",77.1,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""cranageTypes"":[],""extinguishing...","{""areaUnitType"":""sotka"",""area"":6.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-26


In [4]:
print(data2.shape)
data2.head()

(963100, 35)


,id,floornumber,category,bargainterms,description,totalarea,geo,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasgarage,hasshower,bedroomscount,hasdishwasher,repairtype,petsallowed,hasbathhouse,hasfridge,wclocationtype,kitchenarea,roomarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,ptn_dadd
0,275092765,3.0,officeRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...","Описание Бизнес-центра: Деловой центр ""Салова ...",54.5,"{""countryId"":138,""undergrounds"":[{""transportTy...","{""name"":""Салова 45"",""parking"":{""priceMonthly"":...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-25
1,275118966,3.0,flatRent,"{""clientFee"":50,""leaseTermType"":""longTerm"",""pr...",Сдаётся светлая просторная 2-к квартира на дли...,45.0,"{""countryId"":138,""undergrounds"":[{""transportTy...","{""materialType"":""brick"",""hasGarbageChute"":fals...",{},True,2.0,False,True,True,True,True,NaN,False,NaN,False,euro,True,NaN,True,NaN,6.0,NaN,False,NaN,1.0,NaN,25.0,rooms,NaN,2022-06-25
2,206591333,NaN,houseSale,"{""price"":250000.0,""currency"":""rur"",""priceType""...",Дом находится в селе Поника в 7 км. от посёлка...,45.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""wood"",""parking"":{},""floorsCou...","{""status"":""privateFarm"",""areaUnitType"":""sotka""...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outdoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-25
3,240796058,NaN,houseSale,"{""price"":3450000.0,""currency"":""rur"",""priceType...",Продам земельный участок с домом в Советском р...,64.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""wood"",""parking"":{},""floorsCou...","{""status"":""individualHousingConstruction"",""are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,indoors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-25
4,268872358,3.0,flatSale,"{""price"":2650000.0,""currency"":""rur"",""priceType...",Продается двухкомнатная квартира в Приокском!\...,44.8,"{""countryId"":138,""undergrounds"":[],""calculated...","{""materialType"":""brick"",""parking"":{},""floorsCo...",{},NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,design,NaN,NaN,NaN,NaN,6.5,NaN,False,NaN,1.0,NaN,30.0,rooms,1.0,2022-06-25


In [4]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 35 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   id                   1000000 non-null  int64  
 1   floornumber          711355 non-null   float64
 2   category             1000000 non-null  object 
 3   bargainterms         1000000 non-null  object 
 4   description          1000000 non-null  object 
 5   totalarea            934480 non-null   float64
 6   geo                  1000000 non-null  object 
 7   building             1000000 non-null  object 
 8   land                 1000000 non-null  object 
 9   hasfurniture         209850 non-null   object 
 10  roomscount           620398 non-null   float64
 11  hasconditioner       114036 non-null   object 
 12  haskitchenfurniture  203774 non-null   object 
 13  hastv                170207 non-null   object 
 14  haswasher            205488 non-null   object 
 15 

Удаляем столбцы, в которых более 90% пропусков

In [6]:
#data = pd.concat([data1, data2]) # не тестовый вариант
data = data1.iloc[:10000] # тестовый вариант для ускорения
ID = np.array(data['id']) # сохраним id объявок

def get_nan_cols(df, nan_percent=0.8):
    """
        функция для определения колонок, в которых много NaN
    """
    threshold = len(df.index) * nan_percent
    return [c for c in df.columns if sum(df[c].isnull()) >= threshold]

colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
data = data.drop(['ptn_dadd'], axis=1) # неинформативный признак даты, удаляем
data.tail()

,id,floornumber,category,bargainterms,description,totalarea,geo,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasshower,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount
9995,275114190,2.0,flatRent,"{""clientFee"":50,""leaseTermType"":""longTerm"",""pr...","- ЦЕНА СНИЖЕНА!!!!\n- Оперативный показ, ЗАСЕЛ...",65.0,"{""countryId"":138,""undergrounds"":[{""transportTy...","{""materialType"":""brick"",""hasGarbageChute"":true...",{},True,2.0,True,True,True,True,False,True,NaN,euro,True,True,NaN,10.0,False,1.0,NaN,20-15,35.0,rooms,1.0
9996,263410863,4.0,flatSale,"{""price"":2900000.0,""currency"":""rur"",""priceType...","Продам 3к квартиру 4/5эт , 53/42/6кв.м., с/у с...",53.0,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""floorsCount"":5,""cranageTypes"":[...",{},NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,39.0,rooms,NaN
9997,270927262,8.0,flatSale,"{""price"":1.07496E8,""currency"":""rur"",""priceType...",ID 6239. Сколково Парк. Показ в любое время! П...,149.3,"{""countryId"":138,""undergrounds"":[{""transportTy...","{""cargoLiftsCount"":0,""parking"":{},""passengerLi...",{},NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,False,0.0,2.0,NaN,NaN,rooms,0.0
9998,271519208,NaN,houseSale,"{""price"":1800000.0,""currency"":""rur"",""priceType...","ID-37279. Продается летний домик, с возможност...",26.4,"{""countryId"":138,""undergrounds"":[],""calculated...","{""parking"":{},""floorsCount"":1,""cranageTypes"":[...","{""areaUnitType"":""sotka"",""area"":5.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9999,271674201,6.0,flatSale,"{""price"":1.999E7,""currency"":""rur"",""priceType"":...",Продаётся видовая квартира в доме серии П44-т....,76.6,"{""countryId"":138,""undergrounds"":[{""transportTy...","{""materialType"":""monolith"",""hasGarbageChute"":t...",{},NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cosmetic,NaN,NaN,NaN,13.0,False,NaN,2.0,19-15-12,46.0,rooms,NaN


In [7]:
data['geo'].iloc[0]

'{"countryId":138,"undergrounds":[{"transportType":"walk","name":"Кропоткинская","lineColor":"CF0000","time":9,"lineId":10,"id":56,"isDefault":true},{"transportType":"walk","name":"Арбатская","lineColor":"03238B","time":10,"lineId":1,"id":8,"isDefault":false},{"transportType":"walk","name":"Смоленская","lineColor":"03238B","time":11,"lineId":1,"id":115,"isDefault":false}],"calculatedUndergrounds":[{"transportType":"transport","time":2,"distance":1004,"id":8},{"transportType":"walk","time":10,"distance":872,"id":8},{"transportType":"walk","time":9,"distance":789,"id":56},{"transportType":"transport","time":2,"distance":984,"id":56},{"transportType":"walk","time":11,"distance":966,"id":115},{"transportType":"transport","time":4,"distance":1532,"id":115}],"coordinates":{"lat":55.748665,"lng":37.59383},"highways":[],"railways":[{"id":693,"directionIds":[14],"distance":3.0,"time":6,"name":"Москва (Киевский вокзал)","travelType":"byCar"},{"id":693,"directionIds":[14],"distance":2.0,"time":30

### Преобразуем признак geo

In [8]:
def get_geo(arg):
    """
        get countryId
    """
    return eval(arg.replace('false', 'False').replace('true', 'True'))['countryId']

def get_lat(arg):
    """
        get lat
    """
    return eval(arg.replace('false', 'False').replace('true', 'True'))['coordinates']['lat']

def get_lng(arg):
    """
        get lng
    """
    return eval(arg.replace('false', 'False').replace('true', 'True'))['coordinates']['lng']

data['countryId'] = data['geo'].apply(get_geo)
data['lat'] = data['geo'].apply(get_lat)
data['lng'] = data['geo'].apply(get_lng)
data = data.drop(['geo'], axis=1)
data.head()

,id,floornumber,category,bargainterms,description,totalarea,building,land,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasshower,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng
0,243455495,NaN,garageSale,"{""price"":3700000.0,""currency"":""rur"",""priceType...",В районе улицы Старый Арбат в клубном доме пре...,12.5,"{""parking"":{""type"":""underground""},""cranageType...",{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830
1,270527203,2.0,dailyRoomRent,"{""price"":500.0,""currency"":""rur"",""deposit"":0,""b...",Сдается посуточно комфортабельный номер (18м2)...,NaN,"{""materialType"":""monolithBrick"",""parking"":{""ty...",{},True,6.0,True,NaN,True,NaN,NaN,True,NaN,design,NaN,True,NaN,NaN,NaN,NaN,1.0,NaN,NaN,rooms,1.0,138,45.042789,35.377426
2,270569384,7.0,flatRent,"{""clientFee"":35,""leaseTermType"":""longTerm"",""pr...",Сдам квартиру в хорошем состоянии,38.0,"{""cargoLiftsCount"":0,""materialType"":""panel"",""h...",{},True,1.0,NaN,True,NaN,True,NaN,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,138,52.022010,47.842987
3,271459928,4.0,flatRent,"{""clientFee"":0,""leaseTermType"":""longTerm"",""pri...",Сдам новую 1-комнатную квартиру в Болховском м...,42.0,"{""name"":""ЖК «Мкр. Болховский»"",""parking"":{},""f...",{},NaN,1.0,NaN,NaN,NaN,True,NaN,NaN,NaN,cosmetic,NaN,True,NaN,NaN,False,NaN,1.0,NaN,NaN,rooms,NaN,138,52.997077,36.064897
4,272357233,NaN,houseSale,"{""price"":3500000.0,""currency"":""rur"",""priceType...","Продам дачу в снт Тепличный-1, участок площадь...",77.1,"{""parking"":{},""cranageTypes"":[],""extinguishing...","{""areaUnitType"":""sotka"",""area"":6.0}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,54.819368,73.548189


### Преобразуем признаки-словари bargainterms, building, land

In [9]:
# пример работы конструкции из кода функции ниже
data['bargainterms']
keys = set()
for _, row in data.iterrows():
    try:
        dct = eval(row['bargainterms'])
        keys = keys.union({*list(dct.keys())})
    except:
        pass
keys

{'actionId',
 'agentBonus',
 'agentFee',
 'clientFee',
 'contractType',
 'currency',
 'deposit',
 'includedOptions',
 'leaseTermType',
 'minLeaseTerm',
 'paymentPeriod',
 'prepayMonths',
 'price',
 'priceType',
 'saleType',
 'securityDeposit',
 'tenantsType'}

In [10]:
def make_set_keys(data, col):
    """
        Составляет новые колонки из ключей словаря в этой колонке
        data - датасет, в который подставлять колонки
        col - имя колонки, из которой выделяем новые признаки
        Возвращает преобразованный датасет
    """
    # создаём список ключей
    keys = set()
    for _, row in data.iterrows():
        #try:
            dct = eval(row[col].replace('false', 'False').replace('true', 'True'))
            keys = keys.union({*list(dct.keys())})
        #except:
            #pass
    # заполняем соответственно колонки
    for key in keys:
        tmp_arr = []
        for _, row in data.iterrows():
            #try:
            dct = eval(row[col].replace('false', 'False').replace('true', 'True'))
            if key in dct.keys():
                tmp_arr.append(dct[key])
            else:
                tmp_arr.append(np.nan)
            #except:
                #tmp_arr.append(np.nan)
        data[key] = tmp_arr
        
    return data

In [11]:
data = make_set_keys(data, 'bargainterms')
data = make_set_keys(data, 'building')
data = make_set_keys(data, 'land')
data = data.drop(['bargainterms', 'building', 'land'], axis=1)
data.head()

,id,floornumber,category,description,totalarea,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasshower,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng,mortgageAllowed,clientFee,paymentPeriod,actionId,minLeaseTerm,currency,leaseType,leaseTermType,vatType,price,bargainConditions,bargainAllowed,prepayMonths,agentBonus,utilitiesTerms,securityDeposit,agentFee,bargainPrice,priceType,contractType,vatIncluded,vatPrice,tenantsType,deposit,saleType,hasGracePeriod,includedOptions,extinguishingSystemType,cargoLiftsCount,developer,classType,cranageTypes,extinguishingSystemTypes,buildYear,totalArea,shoppingCenterScaleType,parking,liftTypes,tenants,hasGarbageChute,houseLineType,heatingType,name,openingHours,conditioningType,type,ventilationType,materialType,passengerLiftsCount,gatesType,columnGrid,infrastructure,floorsCount,deadline,statusType,series,workingDaysType,accessType,managementCompany,ceilingHeight,possibleToChangeStatus,area,areaUnitType,status
0,243455495,NaN,garageSale,В районе улицы Старый Арбат в клубном доме пре...,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830,NaN,NaN,NaN,NaN,NaN,rur,NaN,NaN,NaN,3700000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,{'type': 'underground'},[],NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'hasEntryByPass': True, 'hasAutomaticGates': ...",NaN,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,270527203,2.0,dailyRoomRent,Сдается посуточно комфортабельный номер (18м2)...,NaN,True,6.0,True,NaN,True,NaN,NaN,True,NaN,design,NaN,True,NaN,NaN,NaN,NaN,1.0,NaN,NaN,rooms,1.0,138,45.042789,35.377426,NaN,NaN,NaN,NaN,NaN,rur,NaN,NaN,NaN,500.0,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,all,NaN,NaN,NaN,any,0.0,NaN,NaN,[],NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,{'type': 'ground'},[],NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,NaN,monolithBrick,NaN,NaN,NaN,{},4.0,{},NaN,NaN,NaN,NaN,NaN,2.8,NaN,NaN,NaN,NaN
2,270569384,7.0,flatRent,Сдам квартиру в хорошем состоянии,38.0,True,1.0,NaN,True,NaN,True,NaN,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,138,52.022010,47.842987,NaN,35.0,monthly,NaN,NaN,rur,NaN,longTerm,NaN,11000.0,NaN,NaN,1.0,NaN,"{'includedInPrice': False, 'price': 0.0, 'flow...",NaN,0.0,NaN,all,NaN,NaN,NaN,NaN,0.0,NaN,NaN,[],NaN,0.0,NaN,NaN,[],[],2004.0,38.0,NaN,{},[],NaN,True,NaN,NaN,NaN,{},NaN,NaN,NaN,panel,1.0,NaN,NaN,{},9.0,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,271459928,4.0,flatRent,Сдам новую 1-комнатную квартиру в Болховском м...,42.0,NaN,1.0,NaN,NaN,NaN,True,NaN,NaN,NaN,cosmetic,NaN,True,NaN,NaN,False,NaN,1.0,NaN,NaN,rooms,NaN,138,52.997077,36.064897,NaN,0.0,monthly,NaN,NaN,rur,NaN,longTerm,NaN,12000.0,NaN,NaN,1.0,NaN,"{'includedInPrice': False, 'price': 2000.0, 'f...",NaN,0.0,NaN,all,NaN,NaN,NaN,NaN,0.0,NaN,NaN,[],NaN,NaN,NaN,NaN,[],[],NaN,42.0,NaN,{},[],NaN,NaN,NaN,NaN,ЖК «Мкр. Болховский»,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},16.0,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,272357233,NaN,houseSale,"Продам дачу в снт Тепличный-1, участок площадь...",77.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,54.819368,73.548189,NaN,NaN,NaN,NaN,NaN,rur,NaN,NaN,NaN,3500000.0,NaN,NaN,NaN,NaN,"{'includedInPrice': False, 'price': 0.0, 'flow...",NaN,NaN,NaN,all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN,[],[],NaN,77.1,NaN,{},[],NaN,NaN,NaN,NaN,NaN,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,{},NaN,{},NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,sotka,NaN


In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 93 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        10000 non-null  int64  
 1   floornumber               7051 non-null   float64
 2   category                  10000 non-null  object 
 3   description               10000 non-null  object 
 4   totalarea                 9368 non-null   float64
 5   hasfurniture              2142 non-null   object 
 6   roomscount                6131 non-null   float64
 7   hasconditioner            1165 non-null   object 
 8   haskitchenfurniture       2091 non-null   object 
 9   hastv                     1754 non-null   object 
 10  haswasher                 2110 non-null   object 
 11  hasbathtub                1546 non-null   object 
 12  hasshower                 1048 non-null   object 
 13  bedroomscount             1464 non-null   float64
 14  repairt

In [13]:
# почистим колонки с большим кол-вом пропусков и удалим ненужные словарные
colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
#data = data.drop(['bargainterms', 'geo', 'building', 'land'], axis=1)
display(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 63 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        10000 non-null  int64  
 1   floornumber               7051 non-null   float64
 2   category                  10000 non-null  object 
 3   description               10000 non-null  object 
 4   totalarea                 9368 non-null   float64
 5   hasfurniture              2142 non-null   object 
 6   roomscount                6131 non-null   float64
 7   hasconditioner            1165 non-null   object 
 8   haskitchenfurniture       2091 non-null   object 
 9   hastv                     1754 non-null   object 
 10  haswasher                 2110 non-null   object 
 11  hasbathtub                1546 non-null   object 
 12  hasshower                 1048 non-null   object 
 13  bedroomscount             1464 non-null   float64
 14  repairt

None

,id,floornumber,category,description,totalarea,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasshower,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng,mortgageAllowed,clientFee,paymentPeriod,currency,leaseTermType,price,prepayMonths,agentBonus,utilitiesTerms,agentFee,priceType,deposit,saleType,includedOptions,cargoLiftsCount,cranageTypes,extinguishingSystemTypes,buildYear,totalArea,parking,liftTypes,hasGarbageChute,heatingType,name,openingHours,materialType,passengerLiftsCount,infrastructure,floorsCount,deadline,ceilingHeight,area,areaUnitType,status
0,243455495,NaN,garageSale,В районе улицы Старый Арбат в клубном доме пре...,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830,NaN,NaN,NaN,rur,NaN,3700000.0,NaN,NaN,NaN,NaN,all,NaN,NaN,[],NaN,[],[],NaN,NaN,{'type': 'underground'},[],NaN,NaN,NaN,{},NaN,NaN,"{'hasEntryByPass': True, 'hasAutomaticGates': ...",NaN,{},NaN,NaN,NaN,NaN
1,270527203,2.0,dailyRoomRent,Сдается посуточно комфортабельный номер (18м2)...,NaN,True,6.0,True,NaN,True,NaN,NaN,True,NaN,design,NaN,True,NaN,NaN,NaN,NaN,1.0,NaN,NaN,rooms,1.0,138,45.042789,35.377426,NaN,NaN,NaN,rur,NaN,500.0,NaN,NaN,NaN,NaN,all,0.0,NaN,[],NaN,[],[],NaN,NaN,{'type': 'ground'},[],NaN,NaN,NaN,{},monolithBrick,NaN,{},4.0,{},2.8,NaN,NaN,NaN
2,270569384,7.0,flatRent,Сдам квартиру в хорошем состоянии,38.0,True,1.0,NaN,True,NaN,True,NaN,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,138,52.022010,47.842987,NaN,35.0,monthly,rur,longTerm,11000.0,1.0,NaN,"{'includedInPrice': False, 'price': 0.0, 'flow...",0.0,all,0.0,NaN,[],0.0,[],[],2004.0,38.0,{},[],True,NaN,NaN,{},panel,1.0,{},9.0,{},NaN,NaN,NaN,NaN
3,271459928,4.0,flatRent,Сдам новую 1-комнатную квартиру в Болховском м...,42.0,NaN,1.0,NaN,NaN,NaN,True,NaN,NaN,NaN,cosmetic,NaN,True,NaN,NaN,False,NaN,1.0,NaN,NaN,rooms,NaN,138,52.997077,36.064897,NaN,0.0,monthly,rur,longTerm,12000.0,1.0,NaN,"{'includedInPrice': False, 'price': 2000.0, 'f...",0.0,all,0.0,NaN,[],NaN,[],[],NaN,42.0,{},[],NaN,NaN,ЖК «Мкр. Болховский»,{},NaN,NaN,{},16.0,{},NaN,NaN,NaN,NaN
4,272357233,NaN,houseSale,"Продам дачу в снт Тепличный-1, участок площадь...",77.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,54.819368,73.548189,NaN,NaN,NaN,rur,NaN,3500000.0,NaN,NaN,"{'includedInPrice': False, 'price': 0.0, 'flow...",NaN,all,NaN,NaN,[],NaN,[],[],NaN,77.1,{},[],NaN,NaN,NaN,{},NaN,NaN,{},NaN,{},NaN,6.0,sotka,NaN


In [14]:
print(data[data['deadline'] == {}]['deadline'].count())
print(data[data['openingHours'] == {}]['openingHours'].count())
print(data.shape)

9215
9979
(10000, 63)


In [15]:
data = data.drop(['deadline', 'openingHours'], axis=1)

In [17]:
# нужно переименовать ключ price в utilitiesTerms и type в parking
def rename_price(arg):
    if arg is np.nan:
        return arg
    try:
        arg['comm_price'] = arg['price']
        arg.pop('price')
        return arg
    except:
        pass
data['utilitiesTerms'].apply(rename_price)

def rename_type(arg):
    if arg is np.nan:
        return arg
    try:
        arg['parking_type'] = arg['type']
        arg.pop('type')
        return arg
    except:
        pass
data['parking'].apply(rename_type)

0       {'parking_type': 'underground'}
1            {'parking_type': 'ground'}
2                                  None
3                                  None
4                                  None
                     ...               
9995                               None
9996                               None
9997                               None
9998                               None
9999                               None
Name: parking, Length: 10000, dtype: object

In [18]:
print(data['utilitiesTerms'].iloc[2])
print(data['parking'].iloc[2])

{'includedInPrice': False, 'flowMetersNotIncludedInPrice': True, 'comm_price': 0.0}
{}


In [19]:
# ещё словарные признаки - utilitiesTerms, infrastructure, parking
# с распаковкой из списка - ничего не осталось

def make_set_keys1(data, col):
    """
        Отличие от make_set_keys в том, что здесь нет
        преобразования к словарю из строки (т.к. уже получаем на вход словарь)
    """
    # создаём список ключей
    keys = set()
    for _, row in data.iterrows():
        try:
            #dct = eval(row[col].replace('false', 'False').replace('true', 'True'))
            keys = keys.union({*list(row[col].keys())})
        except:
            pass
    # заполняем соответственно колонки
    for key in keys:
        tmp_arr = []
        for _, row in data.iterrows():
            try:
                #dct = eval(row[col].replace('false', 'False').replace('true', 'True'))
                if key in row[col].keys():
                    tmp_arr.append(row[col][key])
                else:
                    tmp_arr.append(np.nan)
            except:
                tmp_arr.append(np.nan)
        data[key] = tmp_arr
        
    return data

data = make_set_keys1(data, 'parking')
data = make_set_keys1(data, 'infrastructure')
#data = make_set_keys1(data, 'deadline')
#data = make_set_keys1(data, 'openingHours')
data = make_set_keys1(data, 'utilitiesTerms')
display(data.head())
data.info()

,id,floornumber,category,description,totalarea,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasshower,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng,mortgageAllowed,clientFee,paymentPeriod,currency,leaseTermType,price,prepayMonths,agentBonus,utilitiesTerms,agentFee,priceType,deposit,saleType,includedOptions,cargoLiftsCount,cranageTypes,extinguishingSystemTypes,buildYear,totalArea,parking,liftTypes,hasGarbageChute,heatingType,name,materialType,passengerLiftsCount,infrastructure,floorsCount,ceilingHeight,area,areaUnitType,status,parking_type,locationType,purposeType,placesCount,priceMonthly,isFree,hasInspectionPit,hasPharmacy,hasPool,hasAquapark,hasEntryByPass,hasClothesStudio,hasMinimarket,hasRink,hasHotel,hasShoppingArea,hasFitnessCentre,hasTire,hasCinema,hasFoodCourt,hasCanteen,hasWarehouse,hasNotaryOffice,hasRestaurant,hasBowling,hasVideoSurveillance,hasAtm,hasAutomaticGates,hasHourSecurity,hasExhibitionAndWarehouseComplex,hasBeautyShop,hasConferenceRoom,hasCentralReception,hasCafe,hasSlotMachines,hasMedicalCenter,hasBasement,hasPark,hasGameRoom,hasBuffet,hasBankDepartment,hasSupermarket,hasBabySitting,hasOfficeSpace,hasStudio,hasCarService,hasCarWash,comm_price,flowMetersNotIncludedInPrice,includedInPrice
0,243455495,NaN,garageSale,В районе улицы Старый Арбат в клубном доме пре...,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830,NaN,NaN,NaN,NaN,NaN,3700000.0,NaN,NaN,NaN,NaN,all,NaN,NaN,[],NaN,[],[],NaN,NaN,{'parking_type': 'underground'},[],NaN,NaN,NaN,NaN,NaN,"{'hasEntryByPass': True, 'hasAutomaticGates': ...",NaN,NaN,NaN,NaN,NaN,underground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,True,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,270527203,2.0,dailyRoomRent,Сдается посуточно комфортабельный номер (18м2)...,NaN,True,6.0,True,NaN,True,NaN,NaN,True,NaN,design,NaN,True,NaN,NaN,NaN,NaN,1.0,NaN,NaN,rooms,1.0,138,45.042789,35.377426,NaN,NaN,NaN,NaN,NaN,500.0,NaN,NaN,NaN,NaN,all,0.0,NaN,[],NaN,[],[],NaN,NaN,{'parking_type': 'ground'},[],NaN,NaN,NaN,monolithBrick,NaN,{},4.0,2.8,NaN,NaN,NaN,ground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,270569384,7.0,flatRent,Сдам квартиру в хорошем состоянии,38.0,True,1.0,NaN,True,NaN,True,NaN,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,138,52.022010,47.842987,NaN,35.0,monthly,NaN,longTerm,11000.0,1.0,NaN,"{'includedInPrice': False, 'flowMetersNotInclu...",0.0,all,0.0,NaN,[],0.0,[],[],2004.0,38.0,{},[],True,NaN,NaN,panel,1.0,{},9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,True,False
3,271459928,4.0,flatRent,Сдам новую 1-комнатную квартиру в Болховском м...,42.0,NaN,1.0,NaN,NaN,NaN,True,NaN,NaN,NaN,cosmetic,NaN,True,NaN,NaN,False,NaN,1.0,NaN,NaN,rooms,NaN,138,52.997077,36.064897,NaN,0.0,monthly,NaN,longTerm,12000.0,1.0,NaN,"{'includedInPrice': False, 'flowMetersNotInclu...",0.0,all,0.0,NaN,[],NaN,[],[],NaN,42.0,{},[],NaN,NaN,ЖК «Мкр. Болховский»,NaN,NaN,{},16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.0,True,False
4,272357233,NaN,houseSale,"Продам дачу в снт Тепличный-1, участок площадь...",77.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,54.819368,73.548189,NaN,NaN,NaN,NaN,NaN,3500000.0,NaN,NaN,"{'includ

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 111 entries, id to includedInPrice
dtypes: float64(26), int64(2), object(83)
memory usage: 8.5+ MB


In [20]:
data = data.drop(['parking', 'infrastructure', 'utilitiesTerms'], axis=1)

In [ ]:
"""# отдельная обработка списков словарей
def make_one_dict(arg):
    """
        #функция, которая составляет один словарь из списка словарей
    """
#    if len(arg) == 1 or len(arg) == 0:
#        return arg
    tmp = {}
    for dct in arg:
        #print(arg)
        for k, v in dct.items():
            tmp[k] = v
    return tmp

data['railways'] = data['railways'].apply(make_one_dict)
data['undergrounds'] = data['undergrounds'].apply(make_one_dict)
data['calculatedUndergrounds'] = data['calculatedUndergrounds'].apply(make_one_dict)
data['address'] = data['address'].apply(make_one_dict)
data['district'] = data['district'].apply(make_one_dict)

data = make_set_keys1(data, 'railways')
data = make_set_keys1(data, 'undergrounds')
data = make_set_keys1(data, 'calculatedUndergrounds')
data = make_set_keys1(data, 'address')
data = make_set_keys1(data, 'district')

data = data.drop(['railways', 'undergrounds', 'calculatedUndergrounds', 'address', 'district'], axis=1)"""

In [21]:
def is_empty_list(arg):
    if arg == []:
        return np.nan
data['includedOptions'] = data['includedOptions'].apply(is_empty_list)
data['cranageTypes'] = data['cranageTypes'].apply(is_empty_list)
data['extinguishingSystemTypes'] = data['extinguishingSystemTypes'].apply(is_empty_list)
data['liftTypes'] = data['liftTypes'].apply(is_empty_list)
colss = get_nan_cols(data, 0.9)
data = data.drop(colss, axis=1)
data.head()

,id,floornumber,category,description,totalarea,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasshower,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng,mortgageAllowed,clientFee,paymentPeriod,leaseTermType,price,prepayMonths,agentBonus,agentFee,priceType,deposit,saleType,cargoLiftsCount,buildYear,totalArea,hasGarbageChute,heatingType,name,materialType,passengerLiftsCount,floorsCount,ceilingHeight,area,areaUnitType,status,parking_type,comm_price,includedInPrice
0,243455495,NaN,garageSale,В районе улицы Старый Арбат в клубном доме пре...,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830,NaN,NaN,NaN,NaN,3700000.0,NaN,NaN,NaN,all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underground,NaN,NaN
1,270527203,2.0,dailyRoomRent,Сдается посуточно комфортабельный номер (18м2)...,NaN,True,6.0,True,NaN,True,NaN,NaN,True,NaN,design,NaN,True,NaN,NaN,NaN,NaN,1.0,NaN,NaN,rooms,1.0,138,45.042789,35.377426,NaN,NaN,NaN,NaN,500.0,NaN,NaN,NaN,all,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,monolithBrick,NaN,4.0,2.8,NaN,NaN,NaN,ground,NaN,NaN
2,270569384,7.0,flatRent,Сдам квартиру в хорошем состоянии,38.0,True,1.0,NaN,True,NaN,True,NaN,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,0.0,1.0,NaN,NaN,rooms,0.0,138,52.022010,47.842987,NaN,35.0,monthly,longTerm,11000.0,1.0,NaN,0.0,all,0.0,NaN,0.0,2004.0,38.0,True,NaN,NaN,panel,1.0,9.0,NaN,NaN,NaN,NaN,NaN,0.0,False
3,271459928,4.0,flatRent,Сдам новую 1-комнатную квартиру в Болховском м...,42.0,NaN,1.0,NaN,NaN,NaN,True,NaN,NaN,NaN,cosmetic,NaN,True,NaN,NaN,False,NaN,1.0,NaN,NaN,rooms,NaN,138,52.997077,36.064897,NaN,0.0,monthly,longTerm,12000.0,1.0,NaN,0.0,all,0.0,NaN,NaN,NaN,42.0,NaN,NaN,ЖК «Мкр. Болховский»,NaN,NaN,16.0,NaN,NaN,NaN,NaN,NaN,2000.0,False
4,272357233,NaN,houseSale,"Продам дачу в снт Тепличный-1, участок площадь...",77.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,54.819368,73.548189,NaN,NaN,NaN,NaN,3500000.0,NaN,NaN,NaN,all,NaN,NaN,NaN,NaN,77.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,sotka,NaN,NaN,0.0,False


### Преобразование категориальных признаков

In [22]:
# выделим категориальные и числовые признаки
types = data.dtypes
numerical_cols = []
cat_cols = []
for i in range(len(types)):
    if types[i] != 'object':
        numerical_cols.append(data.columns[i])
    else:
        cat_cols.append(data.columns[i])
#print(types)
#print(numerical_cols)
#print(cat_cols)

In [23]:
data[cat_cols].head()

,category,description,hasfurniture,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasshower,repairtype,petsallowed,hasfridge,wclocationtype,isapartments,allroomsarea,flattype,mortgageAllowed,paymentPeriod,leaseTermType,agentBonus,priceType,saleType,hasGarbageChute,heatingType,name,materialType,areaUnitType,status,parking_type,includedInPrice
0,garageSale,В районе улицы Старый Арбат в клубном доме пре...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,all,NaN,NaN,NaN,NaN,NaN,NaN,NaN,underground,NaN
1,dailyRoomRent,Сдается посуточно комфортабельный номер (18м2)...,True,True,NaN,True,NaN,NaN,True,design,NaN,True,NaN,NaN,NaN,rooms,NaN,NaN,NaN,NaN,all,NaN,NaN,NaN,NaN,monolithBrick,NaN,NaN,ground,NaN
2,flatRent,Сдам квартиру в хорошем состоянии,True,NaN,True,NaN,True,NaN,NaN,cosmetic,False,True,NaN,NaN,NaN,rooms,NaN,monthly,longTerm,NaN,all,NaN,True,NaN,NaN,panel,NaN,NaN,NaN,False
3,flatRent,Сдам новую 1-комнатную квартиру в Болховском м...,NaN,NaN,NaN,NaN,True,NaN,NaN,cosmetic,NaN,True,NaN,False,NaN,rooms,NaN,monthly,longTerm,NaN,all,NaN,NaN,NaN,ЖК «Мкр. Болховский»,NaN,NaN,NaN,NaN,False
4,houseSale,"Продам дачу в снт Тепличный-1, участок площадь...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,all,NaN,NaN,NaN,NaN,NaN,sotka,NaN,NaN,False


In [24]:
data[numerical_cols].head()

,id,floornumber,totalarea,roomscount,bedroomscount,kitchenarea,loggiascount,balconiescount,livingarea,combinedwcscount,countryId,lat,lng,clientFee,price,prepayMonths,agentFee,deposit,cargoLiftsCount,buildYear,totalArea,passengerLiftsCount,floorsCount,ceilingHeight,area,comm_price
0,243455495,NaN,12.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,55.748665,37.593830,NaN,3700000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,270527203,2.0,NaN,6.0,NaN,NaN,NaN,1.0,NaN,1.0,138,45.042789,35.377426,NaN,500.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,4.0,2.8,NaN,NaN
2,270569384,7.0,38.0,1.0,NaN,NaN,0.0,1.0,NaN,0.0,138,52.022010,47.842987,35.0,11000.0,1.0,0.0,0.0,0.0,2004.0,38.0,1.0,9.0,NaN,NaN,0.0
3,271459928,4.0,42.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,138,52.997077,36.064897,0.0,12000.0,1.0,0.0,0.0,NaN,NaN,42.0,NaN,16.0,NaN,NaN,2000.0
4,272357233,NaN,77.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138,54.819368,73.548189,NaN,3500000.0,NaN,NaN,NaN,NaN,NaN,77.1,NaN,NaN,NaN,6.0,0.0


In [25]:
# обработка пропусков
# для числовых признаков
def fillna_num(data_col):
    return data_col.fillna(data_col.mean())

# для категориальных
def fillna_cat(data_col):
    return data_col.fillna('-1-')

for col in numerical_cols:
    data[col] = fillna_num(data[col])
    
for col in cat_cols:
    data[col] = fillna_cat(data[col])

data.head()

,id,floornumber,category,description,totalarea,hasfurniture,roomscount,hasconditioner,haskitchenfurniture,hastv,haswasher,hasbathtub,hasshower,bedroomscount,repairtype,petsallowed,hasfridge,wclocationtype,kitchenarea,isapartments,loggiascount,balconiescount,allroomsarea,livingarea,flattype,combinedwcscount,countryId,lat,lng,mortgageAllowed,clientFee,paymentPeriod,leaseTermType,price,prepayMonths,agentBonus,agentFee,priceType,deposit,saleType,cargoLiftsCount,buildYear,totalArea,hasGarbageChute,heatingType,name,materialType,passengerLiftsCount,floorsCount,ceilingHeight,area,areaUnitType,status,parking_type,comm_price,includedInPrice
0,243455495,6.410722,garageSale,В районе улицы Старый Арбат в клубном доме пре...,12.500000,-1-,2.007014,-1-,-1-,-1-,-1-,-1-,-1-,3.237022,-1-,-1-,-1-,-1-,10.724805,-1-,0.532191,0.625771,-1-,31.756417,-1-,0.748728,138,55.748665,37.593830,-1-,31.815544,-1-,-1-,3700000.0,1.025238,-1-,26.960179,all,33456.63099,-1-,0.635748,1993.034272,369.715382,-1-,-1-,-1-,-1-,1.121543,9.934985,2.967427,33.08853,-1-,-1-,underground,644.445465,-1-
1,270527203,2.000000,dailyRoomRent,Сдается посуточно комфортабельный номер (18м2)...,101.155817,True,6.000000,True,-1-,True,-1-,-1-,True,3.237022,design,-1-,True,-1-,10.724805,-1-,0.532191,1.000000,-1-,31.756417,rooms,1.000000,138,45.042789,35.377426,-1-,31.815544,-1-,-1-,500.0,1.025238,-1-,26.960179,all,0.00000,-1-,0.635748,1993.034272,369.715382,-1-,-1-,-1-,monolithBrick,1.121543,4.000000,2.800000,33.08853,-1-,-1-,ground,644.445465,-1-
2,270569384,7.000000,flatRent,Сдам квартиру в хорошем состоянии,38.000000,True,1.000000,-1-,True,-1-,True,-1-,-1-,3.237022,cosmetic,False,True,-1-,10.724805,-1-,0.000000,1.000000,-1-,31.756417,rooms,0.000000,138,52.022010,47.842987,-1-,35.000000,monthly,longTerm,11000.0,1.000000,-1-,0.000000,all,0.00000,-1-,0.000000,2004.000000,38.000000,True,-1-,-1-,panel,1.000000,9.000000,2.967427,33.08853,-1-,-1-,-1-,0.000000,False
3,271459928,4.000000,flatRent,Сдам новую 1-комнатную квартиру в Болховском м...,42.000000,-1-,1.000000,-1-,-1-,-1-,True,-1-,-1-,3.237022,cosmetic,-1-,True,-1-,10.724805,False,0.532191,1.000000,-1-,31.756417,rooms,0.748728,138,52.997077,36.064897,-1-,0.000000,monthly,longTerm,12000.0,1.000000,-1-,0.000000,all,0.00000,-1-,0.635748,1993.034272,42.000000,-1-,-1-,ЖК «Мкр. Болховский»,-1-,1.121543,16.000000,2.967427,33.08853,-1-,-1-,-1-,2000.000000,False
4,272357233,6.410722,houseSale,"Продам дачу в снт Тепличный-1, участок площадь...",77.100000,-1-,2.007014,-1-,-1-,-1-,-1-,-1-,-1-,3.237022,-1-,-1-,-1-,-1-,10.724805,-1-,0.532191,0.625771,-1-,31.756417,-1-,0.748728,138,54.819368,73.548189,-1-,31.815544,-1-,-1-,3500000.0,1.025238,-1-,26.960179,all,33456.63099,-1-,0.635748,1993.034272,77.100000,-1-,-1-,-1-,-1-,1.121543,9.934985,2.967427,6.00000,sotka,-1-,-1-,0.000000,False


In [26]:
data['description'].iloc[0]

'В районе улицы Старый Арбат в клубном доме предлагается к продаже машиноместо в подземном паркинге площадью 12,5 кв.метров.'

In [27]:
# кодирование категориальных кроме description
data_cat = data[cat_cols]
data_cat = data_cat.drop(['description'], axis=1)
enc = OneHotEncoder(handle_unknown='ignore')
cat_enc = enc.fit_transform(data_cat.astype(str)).toarray()
display(cat_enc)
print(cat_enc.shape)

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

(10000, 2471)


In [ ]:
# обработка description
#vectorizer = TfidfVectorizer(max_features=5000)
#descr_vect = vectorizer.fit_transform(data['description']).toarray()
#print(descr_vect.shape)
#display(descr_vect)

In [ ]:
#vectorizer.vocabulary_

In [28]:
# отмасштабируем числовые признаки, кроме id (?)
num_cols = data[numerical_cols]
num_cols = num_cols.drop(['id'], axis=1)
num_cols = scale(np.array(num_cols))
num_cols

array([[ 1.72146174e-16, -2.05140945e-01,  5.59025358e-16, ...,
         1.46538223e-16,  2.66798110e-17, -1.67928312e-16],
       [-8.54883298e-01,  3.28825370e-17,  5.02642405e+00, ...,
        -5.52468156e-02,  2.66798110e-17, -1.67928312e-16],
       [ 1.14213508e-01, -1.46136423e-01, -1.26764193e+00, ...,
         1.46538223e-16,  2.66798110e-17, -9.51918811e-01],
       ...,
       [ 3.08032869e-01,  1.11400962e-01,  2.50879766e+00, ...,
         1.46538223e-16,  2.66798110e-17, -1.67928312e-16],
       [ 1.72146174e-16, -1.72977696e-01,  5.59025358e-16, ...,
         1.46538223e-16, -1.05468206e-01, -1.67928312e-16],
       [-7.96058530e-02, -5.68197737e-02,  1.24998446e+00, ...,
        -1.08042774e-01,  2.66798110e-17, -1.67928312e-16]])

In [49]:
#embeddings = np.hstack((num_cols, cat_enc))
embeddings = np.hstack((ID.reshape((len(ID), 1)), num_cols, cat_enc))
print(embeddings.shape)
embeddings

(10000, 2497)


array([[ 2.43455495e+08,  1.72146174e-16, -2.05140945e-01, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.70527203e+08, -8.54883298e-01,  3.28825370e-17, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.70569384e+08,  1.14213508e-01, -1.46136423e-01, ...,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       ...,
       [ 2.70927262e+08,  3.08032869e-01,  1.11400962e-01, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.71519208e+08,  1.72146174e-16, -1.72977696e-01, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.71674201e+08, -7.96058530e-02, -5.68197737e-02, ...,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [30]:
ID # сохранённый список id

array([243455495, 270527203, 270569384, ..., 270927262, 271519208,
       271674201])

In [35]:
type(embeddings)

numpy.ndarray

### Сохранение в разных форматах

In [50]:
# save as .txt
np.savetxt('embeddings_test10000.txt', embeddings, delimiter = ' ')

In [51]:
# save as sparse matrix
sparse_embeddings = csc_matrix(embeddings)
save_npz('embeddings_test10000.npz', sparse_embeddings)

In [ ]:
# save in binary format


In [41]:
# make dict
embeddings = np.hstack((num_cols, cat_enc))
print(ID.shape, type(ID))
print(embeddings.shape, type(embeddings))
embeddingsID = {}
for i in range(len(ID)):
    embeddingsID[ID[i]] = embeddings[i]
#print(embeddingsID)

(10000,) <class 'numpy.ndarray'>
(10000, 2496) <class 'numpy.ndarray'>
